In [1]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import albumentations
import torchvision
from torchvision import transforms

import tqdm

from src.utils.metrics import accuracy
from src.utils.data_loaders import ImagesDS
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
path_data = '/home/marchevskiy/Data/other'
device = 'cuda'
batch_size = 16

Data

In [4]:
transforms_train = albumentations.Compose([
    # albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3),
    albumentations.RandomRotate90(p=0.3),
    albumentations.Flip(p=0.3),
    # albumentations.ElasticTransform(alpha=0.5, sigma=25, alpha_affine=25, p=0.3),
    # albumentations.RandomBrightness(limit=0.3, p=0.3),
    # albumentations.RandomContrast(limit=0.15, p=0.3),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

transforms_test = albumentations.Compose([
    # albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

In [29]:
ds = ImagesDS(pd.read_csv(path_data+'/train.csv'), path_data, transforms_train, mode='train')
ds_test = ImagesDS(pd.read_csv(path_data+'/test.csv'), path_data, transforms_test, mode='test')

loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)
tloader = D.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=4)

Model

In [5]:
class ResNext50(nn.Module):
    def __init__(self, num_classes=1108):
        super().__init__()
        preloaded = torchvision.models.resnext50_32x4d(pretrained=True)
        preloaded.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        preloaded.fc = nn.Linear(2048, num_classes, bias=True)
        self.features = nn.Sequential(*list(preloaded.children())[:-2])
        self.dp1 = nn.Dropout(0.6)
        self.classifier = nn.Linear(2048, num_classes, bias=True)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.adaptive_avg_pool2d(features, (1, 1)).view(features.size(0), -1)
        out = self.dp1(out)
        out = self.classifier(out)
        return out

In [38]:
model = ResNext50()
model.load_state_dict(torch.load('../../data/resnext50_v1(d05(384)).pth'))
model.to(device);

Learning

In [32]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-4)

epochs = 40
tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    for x, y in tqdm.tqdm_notebook(loader):
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output, device=device)
        target[np.arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    if epoch > 10:
        optimizer.param_groups[0]['lr'] = 15e-4
    if epoch > 20:
        optimizer.param_groups[0]['lr'] = 7.5e-4
    if epoch > 25:
        optimizer.param_groups[0]['lr'] = 3e-4
    if epoch > 35:
        optimizer.param_groups[0]['lr'] = 1e-4
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

KeyboardInterrupt: 

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

epochs = 1
tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    model.train()
    for x, y in tqdm.tqdm_notebook(loader):
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output, device=device)
        target[np.arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

In [ ]:
torch.save(model.state_dict(), '../../data/resnext50_v1(d05(384)).pth')

K-fold

In [6]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_HEPG2 = train_df[train_df['experiment'].isin(['HEPG2-01', 'HEPG2-02', 'HEPG2-03', 'HEPG2-04', 'HEPG2-05',
                                                       'HEPG2-06', 'HEPG2-07'])]

In [7]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_HEPG2, train_df_HEPG2['sirna'])

In [8]:
model = ResNext50()
model.to(device);

In [ ]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_HEPG2.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_HEPG2.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_v1(d05(384)).pth'))

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 20
    for epoch in range(epochs):
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        model.train()
        for x, y in tqdm.tqdm_notebook(loader_train):
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tloss += loss.item() 
            acc += accuracy(output.cpu(), y)
            del loss, output, y, x, target
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            acc += accuracy(output.cpu(), y)
            del output, y, x, target
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_v1(d05(512))_HEPG2_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen


Epoch 1 -> Train Loss: 0.0027, ACC: 64.32%


In [ ]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_U2OS = train_df[train_df['experiment'].isin(['U2OS-01', 'UHUVEC-012OS-02', 'U2OS-03'])]

In [ ]:
n_folds = 4
folds = KFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_U2OS, train_df_U2OS['sirna'])

In [ ]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_U2OS.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_U2OS.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_v1(d05(384)).pth'))

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 20
    for epoch in range(epochs):
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        model.train()
        for x, y in tqdm.tqdm_notebook(loader_train):
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tloss += loss.item() 
            acc += accuracy(output.cpu(), y)
            del loss, output, y, x, target
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            acc += accuracy(output.cpu(), y)
            del output, y, x, target
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_v1(d05(512))_U2OS_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen

In [ ]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_HUVEC = train_df[train_df['experiment'].isin(['HUVEC-01', 'HUVEC-02', 'HUVEC-03', 'HUVEC-04',
                                                     'HUVEC-05', 'HUVEC-06', 'HUVEC-07', 'HUVEC-08',
                                                     'HUVEC-09', 'HUVEC-10', 'HUVEC-11', 'HUVEC-12',
                                                     'HUVEC-13', 'HUVEC-14', 'HUVEC-15', 'HUVEC-16'])]

In [ ]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_HUVEC, train_df_HUVEC['sirna'])

In [ ]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_HUVEC.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_HUVEC.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_v1(d05(384)).pth'))

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 20
    for epoch in range(epochs):
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        model.train()
        for x, y in tqdm.tqdm_notebook(loader_train):
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tloss += loss.item() 
            acc += accuracy(output.cpu(), y)
            del loss, output, y, x, target
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            acc += accuracy(output.cpu(), y)
            del output, y, x, target
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_v1(d05(512))_HUVEC_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen

In [ ]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_RPE = train_df[train_df['experiment'].isin(['RPE-01', 'RPE-02', 'RPE-03', 'RPE-04',
                                                     'RPE-05', 'RPE-06', 'RPE-07'])]

In [ ]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_RPE, train_df_RPE['sirna'])

In [ ]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_RPE.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_RPE.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_v1(d05(384)).pth'))

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 40
    for epoch in range(epochs):
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        model.train()
        for x, y in tqdm.tqdm_notebook(loader_train):
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tloss += loss.item() 
            acc += accuracy(output.cpu(), y)
            del loss, output, y, x, target
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            target = torch.zeros_like(output, device=device)
            target[np.arange(x.size(0)), y] = 1
            acc += accuracy(output.cpu(), y)
            del output, y, x, target
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_v1(d05(512))_RPE_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen

Submission

In [13]:
model.eval()

@torch.no_grad()
def prediction(model, loader):
    outputs = []
    preds = np.empty(0)
    for x, _ in loader: 
        x = x.to(device)
        output = model(x)
        outputs.append(output)
        idx = output.max(dim=-1)[1].cpu().numpy()
        preds = np.append(preds, idx, axis=0)
    return preds, outputs

preds1, outputs1 = prediction(model, tloader)

In [14]:
preds = ((torch.cat(outputs1)+torch.cat(outputs2)+torch.cat(outputs3)+torch.cat(outputs4)) / 4.).max(dim=-1)[1].cpu().numpy()

In [15]:
submission = pd.read_csv(path_data + '/test.csv')
submission['sirna'] = preds.astype(int)
submission.to_csv('submission.csv', index=False, columns=['id_code','sirna'])

---